A small notebook to check and manipulate the lcs obtained with fpbot in the real-time search project

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

lcs_dir = Path('')
bins_dir = Path('')

#ZTF photometry colours
plot_cols = {'green':'#00A037', 'orange':'#FF7F2B', 'red':'#E0282F'}
plot_filt = {'ZTF_g':plot_cols['green'], 'ZTF_r':plot_cols['orange'], 'ZTF_i':plot_cols['red']}

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [79]:
def load_lc(path):
    lc = pd.read_csv(path, header=0, comment='#')
    for band in lc['filter'].unique():
        if band not in plot_filt.keys():
            lc['filter'] = [f'ZTF_{i[-1]}' for i in lc['filter'].values]
    return lc

def load_binned_data(path, binsize, phase):
    bins = pd.read_csv(path/'bins.csv', header=0)
    der_vals = pd.read_csv(path/'derived_values.csv', header=0)
    verds = pd.read_csv(path/'verdicts.csv', header=0)
    fits = {}
    for band in ['g', 'r', 'i']:
        if len(verds[((verds.band.str.contains(band)) & (verds.binsize==binsize) & (verds.phase==phase))]) > 0:
            if verds[((verds.band.str.contains(band)) & (verds.binsize==binsize) &
                      (verds.phase==phase))].a_init.values[0] != ' ':
                fits[f'ZTF_{band}'] = pd.read_csv(path/f'tail_mod_f{band}_b{binsize}_p{phase}.csv', header=0,index_col=0)
            else:
                fits[f'ZTF_{band}'] = None
        else:
            fits[f'ZTF_{band}'] = None
    return bins[((bins.binsize==binsize) & (bins.phase==phase))], der_vals[der_vals.name=='peak_mjd_after_baseline'].val.values[0], fits

def flux2mag(val, err, zp):
    mag = -2.5*np.log10(val) + zp
    mag_err = 2.5*err/(val*np.log(10))
    return mag, mag_err

def plot_lc(lc, name, xlim=None, ylim=(-100, 1000), plot_bins=False, binsize=100, phase=0.0, sigma_uplim=5):
    fig = plt.figure(figsize=(16,8))
    plt.axhline(0, color='k')
    for band in lc['filter'].unique():
        plt.errorbar(lc[lc['filter']==band].obsmjd, lc[lc['filter']==band].ampl,
                     yerr=lc[lc['filter']==band]['ampl.err'], color=plot_filt[band], ls='none', marker='.',
                     label=band)
    plt.title(name)
    plt.xlim(xlim)
    if xlim == None:
        xlim = plt.xlim()
    plt.ylim(ylim)
    plt.xlabel('mjd')
    plt.ylabel('flux')
    plt.legend()
    plt.show()
    if plot_bins:
        plot_lc_bins(name, xlim, (ylim[0]*10, ylim[1]*10), binsize, phase, sigma_uplim)
    return

def plot_lc_bins(name, xlim=None, ylim=(-100, 1000), binsize=100, phase=0.0, sigma_uplim=5):
    bins, peak_mjd, fits = load_binned_data(bins_dir/name, binsize, phase)
    fig = plt.figure(figsize=(16,8))
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.axhline(0, color='k')
    for band in bins.obs_filter.unique():
        #bins
        for row in bins[bins.obs_filter==band].index:
            if bins.loc[row].significance >= sigma_uplim:
                plt.scatter(bins.loc[row].mjd_bin, bins.loc[row].Fratio, marker='.', color=plot_filt[band])
                plt.plot([bins.loc[row].mjd_start, bins.loc[row].mjd_stop],
                         [bins.loc[row].Fratio, bins.loc[row].Fratio],
                         color=plot_filt[band])
                plt.fill_between([bins.loc[row].mjd_start, bins.loc[row].mjd_stop],
                                 [bins.loc[row].Fratio-sigma*bins.loc[row].Fratio_std,
                                  bins.loc[row].Fratio-sigma*bins.loc[row].Fratio_std],
                                 [bins.loc[row].Fratio+sigma*bins.loc[row].Fratio_std,
                                  bins.loc[row].Fratio+sigma*bins.loc[row].Fratio_std],
                                 color=plot_filt[band], alpha=0.3)
            else:
                plt.scatter(bins.loc[row].mjd_bin, bins.loc[row].Fratio, marker='.', color=plot_filt[band])
                plt.plot([bins.loc[row].mjd_start, bins.loc[row].mjd_stop],
                         [bins.loc[row].Fratio, bins.loc[row].Fratio],
                         color=plot_filt[band])
                plt.fill_between([bins.loc[row].mjd_start, bins.loc[row].mjd_stop],
                                 [bins.loc[row].Fratio-sigma*bins.loc[row].Fratio_std,
                                  bins.loc[row].Fratio-sigma*bins.loc[row].Fratio_std],
                                 [bins.loc[row].Fratio+sigma*bins.loc[row].Fratio_std,
                                  bins.loc[row].Fratio+sigma*bins.loc[row].Fratio_std],
                                 color=plot_filt[band], alpha=0.3, hatch='\\\\')
        #tail fits
        if fits[band] is not None:
            x = fits[band].x.values
            y = fits[band].y.values
            dy = fits[band].dy.values
            plt.plot(x, y, color=plot_filt[band])
            plt.fill_between(x, y-dy, y+dy, color=c[i], alpha=0.3)
    plt.title(f'{name} bins (size = {binsize}, phase = {phase})')
    plt.xlabel('mjd')
    plt.ylabel('flux')
    plt.show()
    print("Note: flux units don't match with non-binned plot")
    return

def plot_lc_mag(lc, name, xlim=None, ylim=(23, 15), sigma=5, plot_bins=False, binsize=100, phase=0.0,
                sigma_uplim=5):
    fig = plt.figure(figsize=(16,8))
    for band in lc['filter'].unique():
        mag, magerr = flux2mag(lc[((lc['filter']==band) & (lc.ampl>=sigma*lc['ampl.err']))].ampl,
                               lc[((lc['filter']==band) & (lc.ampl>=sigma*lc['ampl.err']))]['ampl.err'],
                               lc[((lc['filter']==band) & (lc.ampl>=sigma*lc['ampl.err']))].magzp)
        plt.errorbar(lc[((lc['filter']==band) & (lc.ampl>=sigma*lc['ampl.err']))].obsmjd, mag,
                     yerr=magerr, color=plot_filt[band], ls='none', marker='.',
                     label=band)
        uplim, er = flux2mag(lc[((lc['filter']==band) & (lc.ampl<sigma*lc['ampl.err']))]['ampl.err'],
                             lc[((lc['filter']==band) & (lc.ampl<sigma*lc['ampl.err']))]['ampl.err'],
                             lc[((lc['filter']==band) & (lc.ampl<sigma*lc['ampl.err']))].magzp)
        plt.scatter(lc[((lc['filter']==band) & (lc.ampl<sigma*lc['ampl.err']))].obsmjd, uplim, marker='v',
                    color=plot_filt[band], alpha=0.2)
    plt.title(name)
    plt.xlim(xlim)
    if xlim == None:
        xlim = plt.xlim()
    plt.ylim(ylim)
    plt.xlabel('mjd')
    plt.ylabel('mag')
    plt.legend()
    plt.show()
    if plot_bins:
        plot_lc_bins_mag(name, xlim, ylim, binsize, phase, sigma_uplim)
    return

def plot_lc_bins_mag(name, xlim=None, ylim=(23, 15), binsize=100, phase=0.0, sigma_uplim=5):
    bins, peak_mjd, fits = load_binned_data(bins_dir/name, binsize, phase)
    fig = plt.figure(figsize=(16,8))
    plt.xlim(xlim)
    plt.ylim(ylim)
    for band in bins.obs_filter.unique():
        #bins
        for row in bins[bins.obs_filter==band].index:
            if bins.loc[row].significance >= sigma_uplim:
                mag, mag_err = flux2mag(bins.loc[row].Fratio, bins.loc[row].Fratio_std, 30)
                plt.scatter(bins.loc[row].mjd_bin, mag, marker='.', color=plot_filt[band])
                plt.plot([bins.loc[row].mjd_start, bins.loc[row].mjd_stop], [mag, mag],
                         color=plot_filt[band])
                plt.fill_between([bins.loc[row].mjd_start, bins.loc[row].mjd_stop],
                                 [mag-sigma*mag_err, mag-sigma*mag_err], [mag+sigma*mag_err, mag+sigma*mag_err],
                                 color=plot_filt[band], alpha=0.3)
            else:
                mag, mag_err = flux2mag(sigma_uplim*bins.loc[row].Fratio_std, bins.loc[row].Fratio_std, 30)
                plt.errorbar([bins.loc[row].mjd_bin], mag, yerr=0.3, lolims=True, fmt='.',
                             color=plot_filt[band])
                plt.plot([bins.loc[row].mjd_start, bins.loc[row].mjd_stop], [mag, mag],
                         color=plot_filt[band])
        #tail fits
        if fits[band] is not None:
            x = fits[band].x.values
            y, dy = flux2mag(fits[band].y.values, fits[band].dy.values, 30)
            plt.plot(x, y, color=plot_filt[band])
            plt.fill_between(x, y-dy, y+dy, color=c[i], alpha=0.3)
    plt.title(f'{name} bins (size = {binsize}, phase = {phase})')
    plt.xlabel('mjd')
    plt.ylabel('mag')
    plt.show()
    return

def show_stats(obj, plot_flux=True, xlim_flux=None, ylim_flux=(-100, 1000), plot_mag=True, xlim_mag=None,
               ylim_mag=(23, 15)):
    lc = load_lc(lcs_dir/(obj+'.csv'))
    if plot_flux:
        plot_lc(lc, obj, xlim=xlim_flux, ylim=ylim_flux)
    if plot_mag:
        plot_lc_mag(lc, obj, xlim=xlim_mag, ylim=ylim_mag)
    print(f'{obj} has {len(lc)} data points')
    for band in ['ZTF_g', 'ZTF_r', 'ZTF_i']:
        if band in lc['filter'].unique():
            print(f'{len(lc[lc["filter"]==band])} in {band}')
    print(f'Last data point is at mjd {lc.obsmjd.max()}')
    return